## Importing dependencies

In [1]:
import pandas as pd
import numpy as np
import statistics

## Constructing dataframe from data

In [2]:
raw_excel= pd.ExcelFile('final_data.xlsx')
df = pd.read_excel(raw_excel)

df = df.iloc[:,:5]
print(df.head())

            Unnamed: 0   Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4
0           PhoneMonth    Recruiter       Job       Phone   Accepted
1  2017-04-01 00:00:00  Recruiter 1      Job A          1          0
2  2017-05-01 00:00:00  Recruiter 1      Job A          1          0
3  2017-06-01 00:00:00  Recruiter 1      Job A          2          0
4  2017-08-01 00:00:00  Recruiter 1      Job A          6          3


# Exploratory Analysis

In [3]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,PhoneMonth,Recruiter,Job,Phone,Accepted
1,2017-04-01 00:00:00,Recruiter 1,Job A,1,0
2,2017-05-01 00:00:00,Recruiter 1,Job A,1,0
3,2017-06-01 00:00:00,Recruiter 1,Job A,2,0
4,2017-08-01 00:00:00,Recruiter 1,Job A,6,3


In [4]:
# since there are unnecessary NaN cols from excel, we want to take those out
cols = ['month', 'recruiter', 'job', 'phone', 'num_accepted?']
df.columns = cols

In [5]:
df = df.iloc[1:]

In [6]:
df.columns

Index(['month', 'recruiter', 'job', 'phone', 'num_accepted?'], dtype='object')

### Recruiter ratios

In [7]:
rec = []
for i in range(1, 6):
    curr_recruiter = f'Recruiter {i}'
    rec.append(df[df['recruiter'] == curr_recruiter])

In [8]:
# if you want to refer to a specific recruiter, rec[recruiter_no-1]
# ie. recruiter 1
rec[0].head()

,month,recruiter,job,phone,num_accepted?
1,2017-04-01 00:00:00,Recruiter 1,Job A,1,0
2,2017-05-01 00:00:00,Recruiter 1,Job A,1,0
3,2017-06-01 00:00:00,Recruiter 1,Job A,2,0
4,2017-08-01 00:00:00,Recruiter 1,Job A,6,3
5,2017-09-01 00:00:00,Recruiter 1,Job A,3,0


In [9]:
# accepted: phonecall ratios
rec_ratios = []
for i in range(5):
    rec_ratios.append(rec[i]['num_accepted?'].sum()/ rec[i]['phone'].sum())

In [10]:
print(f'Recruiter conversion ratios by recruiter (1,2,3,4,5):\n{rec_ratios}\n')
avg_rec_ratio = sum(rec_ratios)/len(rec_ratios) # 5.997%
print(f'Average recruiter ratio (in %): {avg_rec_ratio*100}')

Recruiter conversion ratios by recruiter (1,2,3,4,5):
[0.07096774193548387, 0.056666666666666664, 0.06208425720620843, 0.055408970976253295, 0.05474452554744526]

Average recruiter ratio (in %): 5.99744324664115


In [11]:
# let's show this as a percent because it's pretty low
avg_perc = round(avg_rec_ratio*1000, 2)/10 # so around 6%, that is the conversion ratio


### Job conversion ratios

In [12]:
# let's find the conversion ratios for each job - A,B,C,D
jobs = [df[df['job'] == 'Job {}'.format(chr(ord('A')+i))] for i in range(4)] 

In [13]:
job_conv_ratios = [jobs[i]['num_accepted?'].sum()/jobs[i]['phone'].sum() for i in range(4)]

In [14]:
print(f'Job conversion ratios by job (A,B,C,D): {job_conv_ratios}\n')
avg_job_conv_ratios = round(1000*sum(job_conv_ratios)/len(job_conv_ratios),2)/10
print(f'Average job conversion ratio: {avg_job_conv_ratios}%') # 4.41%, also pretty low


Job conversion ratios by job (A,B,C,D): [0.08045977011494253, 0.0590851334180432, 0.04484304932735426, 0.03608247422680412]

Average job conversion ratio: 5.512%


#### Calculations of std for sensitivity analysis before we construct the model

In [15]:
std_rec_ratios = statistics.stdev(rec_ratios)
print(f'Standard deviation for the recruiter ratios: {std_rec_ratios}') # 0.679%

std_job_conv_ratios = statistics.stdev(job_conv_ratios)
print(f'Standard deviation for the job conversion ratios: {std_job_conv_ratios}') # 1.9%


Standard deviation for the recruiter ratios: 0.006790450888942972
Standard deviation for the job conversion ratios: 0.019372397135133133


### Message success ratios by recruiter

To even make a call, recruiters have to sift through tons of profiles from searches and send messages in order to even select candidates appropriate for a given position

In [16]:
msg_data = pd.read_csv('message_data.csv')
msg_data = msg_data.iloc[:,0:9] # throwing out unnamed NaN cols

In [17]:
msg_data.head()

,Seat holder,Month,Unique daily logins,Profiles viewed,Profiles saved,Searches performed,Messages sent,Messages accepted,Messages declined
0,Recruiter 2,1/1/19,19,1633,101,101,24,1,0
1,Recruiter 4,1/1/19,18,262,267,41,49,19,4
2,Recruiter 3,1/1/19,22,223,156,27,126,18,2
3,Recruiter 5,1/1/19,22,101,185,45,139,46,9
4,Recruiter 1,1/1/19,20,117,57,90,255,41,26


In [18]:
# let's get some summary statistics by recruiter after we change 'Seat holder' to recruiter
msg_data = msg_data.rename(columns={'Seat holder':'recruiter'})

describe_by_recruiter = [msg_data[msg_data['recruiter'] == f'Recruiter {i}'].describe() for i in range(1, 6)]
describe_by_recruiter[0]

,Unique daily logins,Profiles viewed,Profiles saved,Searches performed,Messages sent,Messages accepted,Messages declined
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.00000,7.000000
mean,13.857143,56.857143,19.142857,30.571429,97.142857,20.00000,12.857143
std,4.947342,36.089835,20.538002,30.527895,86.260127,14.21267,12.266874
min,5.000000,6.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,12.500000,39.500000,4.500000,10.000000,37.500000,12.50000,1.500000
50%,14.000000,47.000000,14.000000,28.000000,80.000000,14.00000,11.000000
75%,16.500000,74.500000,27.000000,38.000000,135.000000,30.00000,24.500000
max,20.000000,117.000000,57.000000,90.000000,255.000000,41.00000,27.000000


In [19]:
# dict comprehension to be able to look-up this data by recruiter num
msg_data_by_recruiter = {f'recruiter {v}':msg_data[msg_data['recruiter'] == f'Recruiter {v}'] \
                                                                        for (k,v) in enumerate(range(1, 6))}
msg_data_by_recruiter

{'recruiter 1':       recruiter   Month  Unique daily logins  Profiles viewed  Profiles saved  \
 4   Recruiter 1  1/1/19                   20              117              57   
 5   Recruiter 1  2/1/19                   12               43               8   
 12  Recruiter 1  3/1/19                   19               86              35   
 18  Recruiter 1  4/1/19                   14               63              19   
 20  Recruiter 1  5/1/19                   14               36               1   
 25  Recruiter 1  6/1/19                    5                6               0   
 30  Recruiter 1  7/1/19                   13               47              14   
 
     Searches performed  Messages sent  Messages accepted  Messages declined  
 4                   90            255                 41                 26  
 5                   19             40                 14                  1  
 12                  38             80                 33                 11  
 18        

In [20]:
# what does the structure of msgs sent and accepted look like
print(len(msg_data_by_recruiter['recruiter 1'])) # only 7 lines long per recruiter
print(msg_data_by_recruiter['recruiter 1'][['Messages sent', 'Messages accepted']])

# finding ratios of accepted:sent
perc_of_accepted_msgs = {f'recruiter {v}': msg_data_by_recruiter[f'recruiter {v}']['Messages accepted'].sum() / 
                         msg_data_by_recruiter[f'recruiter {v}']['Messages sent'].sum()
                         for k,v in enumerate(range(1, 6))}
perc_of_accepted_msgs # looks like recruiter 4 has the highest ratio of messages accepted:sent

7
    Messages sent  Messages accepted
4             255                 41
5              40                 14
12             80                 33
18            134                 27
20            136                 11
25              0                  0
30             35                 14


{'recruiter 1': 0.20588235294117646,
 'recruiter 2': 0.13969732246798602,
 'recruiter 3': 0.1875,
 'recruiter 4': 0.33490011750881316,
 'recruiter 5': 0.14949201741654572}

In [21]:
from functools import reduce
msg_rates = [v for k,v in perc_of_accepted_msgs.items()]

avg_msg_accepted_rate = reduce(lambda x, y: x+y, msg_rates)/5
msg_stdev_accepted = statistics.stdev(msg_rates)

print(f'Avg for accepted msgs: {avg_msg_accepted_rate}') # 20.3%
print(f'Std for accepted msgs: {msg_stdev_accepted}') # 7.8%

Avg for accepted msgs: 0.20349436206690424
Std for accepted msgs: 0.07828682866696615


### Constructing the model

Since we know that recruiters take a lot of time finding candidates and sending messages, we can construct an objective function from the ratios we've acquired for each recruiter's call success rate

Our goal is to create a model where recruiters have to spend the least amount of time they need to calling candidates. We don't want to make this a maximization problem and risk burnout from our recruiters. Given what we deem is feasible  from looking at their historical performances, we want to have the most number of accepted offers with the least amount of phone calls.

In [22]:
# looks like our timeframe is monthly 
total_per_month = df[['recruiter','month', 'num_accepted?', 'phone']].groupby(([df['month']])).sum()

avg_accepted_month = statistics.mean(total_per_month['num_accepted?'])
avg_calls_month = statistics.mean(total_per_month['phone'])

# we want to construct constraints that help all recruiters hit these targets
print(f'Average number of accepted offers per month: {avg_accepted_month}')
print(f'Average number of calls per month: {avg_calls_month}')

Average number of accepted offers per month: 2.838235294117647
Average number of calls per month: 47.35294117647059


In [23]:
print(total_per_month['num_accepted?'].describe())
print('\n{}'.format(total_per_month['phone'].describe()))

count    68.000000
mean      2.838235
std       3.074257
min       0.000000
25%       0.000000
50%       2.000000
75%       5.000000
max      10.000000
Name: num_accepted?, dtype: float64

count     68.000000
mean      47.352941
std       32.657318
min        5.000000
25%       16.750000
50%       49.000000
75%       74.000000
max      115.000000
Name: phone, dtype: float64


#### As we see below, there is a high standard deviation for these monthly stats, so they should not be taken as truth, though they do help guide us in constructing our constraints

In [24]:
std_num_accepted = statistics.stdev(total_per_month['num_accepted?'])
std_num_calls = statistics.stdev(total_per_month['phone'])

print(f'Stdev number of accepted offers per month: {std_num_accepted}')
print(f'Stdev calls per month: {std_num_calls}')

Stdev number of accepted offers per month: 3.0742565858676567
Stdev calls per month: 32.65731830664549


#### Coefficients for our objective function:

In [25]:
rec_ratios # --> maximize phone calls: ∑x_i*rec_ratios[i]

[0.07096774193548387,
 0.056666666666666664,
 0.06208425720620843,
 0.055408970976253295,
 0.05474452554744526]

#### Job conversion ratios will help us find constraints (ie. recruiter we want recruiting certain jobs since they're the best at those)

Let's rank our recruiters based on their personal job conv rates

In [26]:
# we initialize an empty dict so we can sort job conversion rates of recruiters by job
rates = {'job {}'.format(chr(ord('A')+k)):v for k,v in enumerate(range(0,4))}

In [27]:
for i in range(4):
    curr_job = jobs[i]
    rates['job {}'.format(chr(ord('A')+i))] = [curr_job[curr_job['recruiter'] == f'Recruiter {j}']['num_accepted?'].sum()/ \
                                               curr_job[curr_job['recruiter'] == f'Recruiter {j}']['phone'].sum() \
                                               for j in range(1,6)]
    


### This is our job coefficients for the constraint matrix

In [28]:
# rows are jobs, columns are recruiters
job_constraint_matrix = pd.DataFrame(rates).T
job_constraint_matrix.columns = [1,2,3,4,5]
job_constraint_matrix

,1,2,3,4,5
job A,0.093023,0.089623,0.073298,0.068966,0.063492
job B,0.059211,0.044374,0.079268,0.048611,0.064516
job C,0.057692,0.056291,0.018182,0.063492,0.000000
job D,0.054054,0.072727,0.000000,0.000000,0.047619


### Now that we are optimizing, we will have min targets be the max number of accepted offers we saw that were accepted for that job in a month 

In [29]:
import math
# rounding each monthly job performance up so we have groups aspire to outperform previous averages
mean_job_performance_per_month = [math.ceil(max(jobs[i][['month', 'num_accepted?']].groupby(['month']).sum()['num_accepted?'])) \
                                  for i in range(4)]
mean_job_performance_per_month

[7, 6, 3, 2]

### We are setting a lower bound on each of the job targets to get our full job constraint matrix

In [30]:
job_constraint_matrix['min_targets'] = mean_job_performance_per_month
job_constraint_matrix

,1,2,3,4,5,min_targets
job A,0.093023,0.089623,0.073298,0.068966,0.063492,7
job B,0.059211,0.044374,0.079268,0.048611,0.064516,6
job C,0.057692,0.056291,0.018182,0.063492,0.000000,3
job D,0.054054,0.072727,0.000000,0.000000,0.047619,2


#### Now, we want to find who performed the best for each position so we can encourage via other constraints, where recruiters should be devoting their phone time

In [31]:
best_recruiters = {k:(f'Recruiter {v.index(max(v)) + 1}',max(v)) for k,v in rates.items()}
print(f'This dictionary indicates who is best at recruiting certain jobs:\n')
best_recruiters

This dictionary indicates who is best at recruiting certain jobs:



{'job A': ('Recruiter 1', 0.09302325581395349),
 'job B': ('Recruiter 3', 0.07926829268292683),
 'job C': ('Recruiter 4', 0.06349206349206349),
 'job D': ('Recruiter 2', 0.07272727272727272)}

### Since there are 4 jobs, if a certain recruiter performs best at a particular role, they will have to make sure that they meet at least 25% of the min target for that position

# Our model:
### Minimize:
z = .0710x1 + .0567x2 + .06208x3 + .0554x4 + .05474x5, where x_i = (x_iA + x_iB + x_iC + x_iD)
### Subject to:
Job Constraint matrix,

x_ij >= .25(min_target_job_j)

x_i >= 0

# Solving:

In [32]:
from pulp import *

x_names = []
for i in range(1, 6):
    x_names.append(['x_{}{}'.format(i,chr(ord('A')+j)) for j in range(4)])

# note that we are already setting the non-negativity condition for the objective function
x = [ LpVariable(x_names[i][j], lowBound = 0, cat = 'continuous') for i in range(5) for j in range(4) ]

In [33]:
## creating an objective function
objective_function = LpAffineExpression([ ( x[4*i+j], rec_ratios[i] ) for i in range(5) for j in range(4) ])
objective_function

0.07096774193548387*x_1A + 0.07096774193548387*x_1B + 0.07096774193548387*x_1C + 0.07096774193548387*x_1D + 0.056666666666666664*x_2A + 0.056666666666666664*x_2B + 0.056666666666666664*x_2C + 0.056666666666666664*x_2D + 0.06208425720620843*x_3A + 0.06208425720620843*x_3B + 0.06208425720620843*x_3C + 0.06208425720620843*x_3D + 0.055408970976253295*x_4A + 0.055408970976253295*x_4B + 0.055408970976253295*x_4C + 0.055408970976253295*x_4D + 0.05474452554744526*x_5A + 0.05474452554744526*x_5B + 0.05474452554744526*x_5C + 0.05474452554744526*x_5D + 0

In [34]:
LP_problem = pulp.LpProblem("Linear Optimization", pulp.LpMinimize)
LP_problem += objective_function, "N"

In [35]:
x

[x_1A,
 x_1B,
 x_1C,
 x_1D,
 x_2A,
 x_2B,
 x_2C,
 x_2D,
 x_3A,
 x_3B,
 x_3C,
 x_3D,
 x_4A,
 x_4B,
 x_4C,
 x_4D,
 x_5A,
 x_5B,
 x_5C,
 x_5D]

In [36]:
## constraints
# taking coefficients from job_constraint_matrix for variables
job_constraintsA = LpAffineExpression([ (x[4*j+0], job_constraint_matrix.iloc[0,j]) for j in range(5) ])
job_constraintsB = LpAffineExpression([ (x[4*j+1], job_constraint_matrix.iloc[1,j]) for j in range(5) ])
job_constraintsC = LpAffineExpression([ (x[4*j+2], job_constraint_matrix.iloc[2,j]) for j in range(5) ])
job_constraintsD = LpAffineExpression([ (x[4*j+3], job_constraint_matrix.iloc[3,j]) for j in range(5) ])

job_coefs_matrix = [job_constraintsA, job_constraintsB, job_constraintsC, job_constraintsD]

In [37]:
# setting min_targets from job_constraint_matrix for the job coefficients matrix (job_coef_matrix)
for i in range(4):
    constraint_name = chr(ord('A')+i)
    LP_problem += LpConstraint(e=job_coefs_matrix[i], # LHS vars
                               sense=1, # GE
                               name=f'coefs {constraint_name}', # name  
                               rhs=job_constraint_matrix.iloc[i,5]) # RHS target

In [38]:
LP_problem += LpConstraint(e=job_coefs_matrix[0], # LHS vars
                               sense=1, # GE
                               rhs=job_constraint_matrix.iloc[0,5]) # RHS target

In [39]:
LpConstraint(e=job_coefs_matrix[1], # LHS vars
                               sense=1, # GE
                               name=f'coefs {constraint_name}', # name  
                               rhs=job_constraint_matrix.iloc[1,5])

0.05921052631578947*x_1B + 0.044374009508716325*x_2B + 0.07926829268292683*x_3B + 0.04861111111111111*x_4B + 0.06451612903225806*x_5B + -6 >= 0

In [40]:
job_constraint_matrix.iloc[1,5]

6

In [41]:
# we want the recruiter with the best proporitions for job {letter}
# to make at least 25% of the min required calls
LP_problem += LpConstraint(e=LpAffineExpression([(x[0], 4) ]),
                           sense=1,
                           name='job A best: min target',
                           rhs=job_constraint_matrix.iloc[0,5])

LP_problem += LpConstraint(e=LpAffineExpression([(x[9], 4) ]),
                           sense=1,
                           name='job B best: min target',
                           rhs=job_constraint_matrix.iloc[1,5])

LP_problem += LpConstraint(e=LpAffineExpression([(x[14], 4) ]),
                           sense=1,
                           name='job C best: min target',
                           rhs=job_constraint_matrix.iloc[2,5])

LP_problem += LpConstraint(e=LpAffineExpression([(x[7], 4) ]),
                           sense=1,
                           name='job D best: min target',
                           rhs=job_constraint_matrix.iloc[3,5])

In [42]:
## none of the recruiters should ever exceed the max phone calls made in a month
max_phone_calls = max(total_per_month['phone'])
for i in range(5):
    curr_rec = LpAffineExpression( [ (x[4*i+j], 1) for j in range(4)])
    LP_problem += LpConstraint(e=curr_rec, # LHS vars
                               sense=-1, # LE
                               name=f'recruiter {i+1} max', # name  
                               rhs=max_phone_calls) # RHS target

In [43]:
## non-negativity constraints
for i in range(20):
    curr_var = LpAffineExpression( [(x[i],1)] )
    LP_problem += LpConstraint(e = curr_var,
                              sense = 1,
                              rhs = 0)

In [44]:
LP_problem.solve()
pulp.LpStatus[LP_problem.status]

# since we can't make anything but integer phone calls
for var in LP_problem.variables():
    print(var.name, math.ceil(var.varValue))

x_1A 2
x_1B 0
x_1C 0
x_1D 0
x_2A 77
x_2B 0
x_2C 0
x_2D 28
x_3A 0
x_3B 76
x_3C 0
x_3D 0
x_4A 0
x_4B 0
x_4C 48
x_4D 0
x_5A 0
x_5B 0
x_5C 0
x_5D 0


In [45]:
if LP_problem.status == 1:
    print('We\'ve reached the optimal solution.')

We've reached the optimal solution.
